In [2]:
pip install pymysql

                                              0.0/44.8 kB ? eta -:--:--
     ---------------------------------------- 44.8/44.8 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass # To get the password without showing the input

password = getpass.getpass("mysql2023")
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

mysql2023 ········


In [7]:
# Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

# engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
# month: an integer representing the month for which rental data is to be retrieved.
# year: an integer representing the year for which rental data is to be retrieved.

In [8]:
def rentals_month(engine, month, year=2005):
    """
    This function retrieves the rental data from the Sakila database for the specified month and year and returns it as a pandas DataFrame.
    
    Parameters:
    - engine: SQLAlchemy engine object. An engine object manages the connection to the database.
    - month: integer value representing the month for which the rental data is required.
    - year: integer value representing the year for which the rental data is required.
    Returns:
    - A pandas DataFrame containing the rental data for the specified month and year.
    """
    # create the SQL query
    query = "SELECT * FROM rental WHERE MONTH(rental_date) = "+str(month)+" AND YEAR(rental_date) = "+str(year)
    
    # execute the query and store the results in a DataFrame
    data = pd.read_sql_query(query, engine)
    return data

In [9]:
may = rentals_month(engine, 5, 2005)
may

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
1151,1153,2005-05-31 21:36:44,2725,506,2005-06-10 01:26:44,2,2006-02-15 21:30:53
1152,1154,2005-05-31 21:42:09,2732,59,2005-06-08 16:40:09,1,2006-02-15 21:30:53
1153,1155,2005-05-31 22:17:11,2048,251,2005-06-04 20:27:11,2,2006-02-15 21:30:53
1154,1156,2005-05-31 22:37:34,460,106,2005-06-01 23:02:34,2,2006-02-15 21:30:53


In [ ]:
# 3. Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

In [11]:
def rental_count_month(df, month, year=2005):
    """
    Takes a DataFrame of rental data as input and returns a new DataFrame
    containing the number of rentals made by each customer_id during the specified
    month and year. The month and year should be passed as numeric parameters.
    
    Args:
    - df: DataFrame of rental data
    - month: int, numeric value of the month (1-12)
    - year: int, numeric value of the year (default 2005)
    
    Returns:
    - rentals_by_customer: DataFrame containing customer_id and rental count
    """
    # Group the rentals by customer_id and count the number of rentals for each customer
    rentals_by_customer = df.loc[(df['rental_date'].dt.month == month) & (df['rental_date'].dt.year == year)] \
                          .groupby('customer_id')['rental_id'].count().reset_index()

    # Rename the 'rental_id' column to 'rentals_{month}'
    rentals_by_customer = rentals_by_customer.rename(columns={'rental_id': f'rentals_{month}_{year}'})

    return rentals_by_customer

# Example usage
# Assuming 'may' is the DataFrame returned by the rentals_month function
customer_rentals_may = rental_count_month(may, 5)
print(customer_rentals_may)

     customer_id  rentals_5_2005
0              1               2
1              2               1
2              3               2
3              5               3
4              6               3
..           ...             ...
515          594               4
516          595               1
517          596               6
518          597               2
519          599               1

[520 rows x 2 columns]


In [12]:
def compare_rentals(df1, df2):
    """
    Compares the number of rentals made by customers in two different months and years
    and returns a combined DataFrame with a 'difference' column.
    
    Args:
    - df1: DataFrame containing rental data for the first month and year
    - df2: DataFrame containing rental data for the second month and year
    
    Returns:
    - combined_df: DataFrame with customer_id, rental count for each month, and the difference
      between the rental counts
    """
    # Merge the two DataFrames on 'customer_id' column
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer')
    
    # Calculate the difference between the rental counts for the two months
    combined_df['difference'] = combined_df[df2.columns[1]] - combined_df[df1.columns[1]]
    
    return combined_df

In [13]:
june = rentals_month(engine,6)
customer_rentals_june = rental_count_month(june, 6)
customer_rentals_june

comparative = compare_rentals(customer_rentals_may, customer_rentals_june)
comparative

,customer_id,rentals_5_2005,rentals_6_2005,difference
0,1,2.0,7.0,5.0
1,2,1.0,1.0,0.0
2,3,2.0,4.0,2.0
3,5,3.0,5.0,2.0
4,6,3.0,4.0,1.0
...,...,...,...,...
593,583,NaN,6.0,NaN
594,585,NaN,4.0,NaN
595,591,NaN,3.0,NaN
596,592,NaN,5.0,NaN
